# Extract the respiration signals from a video

This notebook extracts the raw signals for each fine-tuned EfficientPhys models.

In [1]:
import os

import numpy as np

models_dirs = [
    # Trained 5 epochs with MSE loss
    os.path.join('..', 'models', 'efficeint_phys_fine_tuned', '20240503_174905'),
    # Trained 5 epochs with SoftDWT loss
    os.path.join('..', 'models', 'efficeint_phys_fine_tuned', '20240503_190048'),
]

In [2]:
import respiration.utils as utils

device = utils.get_torch_device()
dim = 72
frame_depth = 20

In [3]:
model_info = []

for model_dir in models_dirs:
    manifest_path = os.path.join(model_dir, 'manifest.json')
    manifest = utils.read_json(manifest_path)

    for tuned_model in manifest['tuned_models']:
        model_info.append({
            'id': manifest['id'],
            'base_model': manifest['base_model'],
            'loss_fn': manifest['loss_fn'],
            'model': tuned_model['model'],
            'epoch': tuned_model['epoch'],
            'validation_loss': tuned_model['validation_loss'],
        })

In [4]:
import pandas as pd

model_df = pd.DataFrame(model_info)
model_df

,id,base_model,loss_fn,model,epoch,validation_loss
0,20240503_174905,EfficientPhys,mse,model_20240503_174905_0.pth,0,0.225236
1,20240503_174905,EfficientPhys,mse,model_20240503_174905_4.pth,4,0.076533
2,20240503_190048,EfficientPhys,soft_dwt,model_20240503_190048_0.pth,0,293.527710
3,20240503_190048,EfficientPhys,soft_dwt,model_20240503_190048_1.pth,1,166.856186


In [5]:
import torch
from respiration.extractor.efficient_phys import EfficientPhys


def load_model(path) -> EfficientPhys:
    # Wrap modul in nn.DataParallel
    model = EfficientPhys(img_size=dim, frame_depth=frame_depth)
    # Fix model loading: Some key have an extra 'module.' prefix
    model = torch.nn.DataParallel(model)
    model.to(device)
    model.load_state_dict(torch.load(path, map_location=device))

    _ = model.eval()

    return model

In [6]:
import respiration.dataset as repository

dataset = repository.from_default()

In [7]:
def predict_signal(model: EfficientPhys, subject: str, setting: str) -> tuple[np.ndarray, int, int]:
    video_path = dataset.get_video_path(subject, setting)
    frame_count = utils.get_frame_count(video_path)

    chunk_size = (frame_count // frame_depth) * frame_depth - (frame_depth - 1)
    # chunk_size = frame_depth * 20 + 1

    frames, meta = utils.read_video_rgb(video_path, chunk_size)
    frames = utils.down_sample_video(frames, dim)

    frames = torch.tensor(frames, dtype=torch.float32, device=device).permute(0, 3, 1, 2)

    with torch.no_grad():
        prediction = model(frames).cpu().detach().numpy().squeeze()
        
    del frames

    return prediction, chunk_size, meta.fps

In [8]:
predictions = []

for model_dir in models_dirs:
    manifest_path = os.path.join(model_dir, 'manifest.json')
    manifest = utils.read_json(manifest_path)

    best_model = manifest['tuned_models'][-1]
    model_path = os.path.join(model_dir, best_model['model'])

    model = load_model(model_path)

    for scenario in manifest['testing_scenarios']:
        if not dataset.contains(scenario[0], scenario[1]):
            print(f'Skipping {scenario[0]} - {scenario[1]}')
            continue

        print(f'Processing {scenario[0]} - {scenario[1]}')
        prediction, chunks, fps = predict_signal(model, scenario[0], scenario[1])

        predictions.append({
            'id': manifest['id'],
            'subject': scenario[0],
            'setting': scenario[1],
            'chunk_size': chunks,
            'fps': fps,
            'prediction': prediction.tolist(),
        })

Processing Proband26 - 101_natural_lighting
Processing Proband26 - 101_natural_lighting


In [9]:
prediction_df = pd.DataFrame(predictions)

evaluation_dir = os.path.join('..', 'evaluation', 'efficeint_phys_fine_tuned')
if not os.path.exists(evaluation_dir):
    os.makedirs(evaluation_dir)

evaluation_path = os.path.join(evaluation_dir, 'predictions.csv')
prediction_df.to_csv(evaluation_path, index=False)

prediction_df

,id,subject,setting,chunk_size,fps,prediction
0,20240503_174905,Proband26,101_natural_lighting,3581,30,"[0.17320716381072998, 0.10809916257858276, 0.1..."
1,20240503_190048,Proband26,101_natural_lighting,3581,30,"[-0.37972742319107056, -0.2880082130432129, -0..."
